In [1]:
pip install qiskit qiskit_aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 5.8 MB/s eta 0:00:00


In [4]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels to binary
encoder = LabelEncoder()
y = encoder.fit_transform((y == 0).astype(int))  # Only classify setosa vs non-setosa
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (120, 4)
Test set shape: (30, 4)


In [5]:
from qiskit import QuantumCircuit
def create_quantum_circuit(data):
    """Creates a quantum circuit to encode classical data."""
    num_features = len(data)
    qc = QuantumCircuit(num_features)

    for i in range(num_features):
        qc.ry(data[i], i)  # Encode data using Ry rotation gates

    return qc
# Example encoding for the first training sample
qc = create_quantum_circuit(X_train[0])
print(qc.draw())

     ┌─────────────┐
q_0: ┤ Ry(-1.5065) ├
     └┬────────────┤
q_1: ─┤ Ry(1.2492) ├
     ┌┴────────────┤
q_2: ┤ Ry(-1.5676) ├
     ├─────────────┤
q_3: ┤ Ry(-1.3154) ├
     └─────────────┘


In [6]:
X_train[0]

array([-1.50652052,  1.24920112, -1.56757623, -1.3154443 ])

In [7]:
from qiskit.circuit import Parameter

def create_variational_circuit(num_qubits):
    """Creates a parameterized variational quantum circuit."""
    qc = QuantumCircuit(num_qubits)
    # Define parameters
    theta = [Parameter(f'θ_{i}') for i in range(num_qubits)]
    for i in range(num_qubits): #applying rotations
        qc.ry(theta[i], i)
    for i in range(num_qubits - 1): #applying CNOT entanglements
        qc.cx(i, i + 1)
    return qc, theta

# Example variational circuit for 4 features
var_qc, params = create_variational_circuit(4)
print(var_qc.draw())

     ┌─────────┐               
q_0: ┤ Ry(θ_0) ├──■────────────
     ├─────────┤┌─┴─┐          
q_1: ┤ Ry(θ_1) ├┤ X ├──■───────
     ├─────────┤└───┘┌─┴─┐     
q_2: ┤ Ry(θ_2) ├─────┤ X ├──■──
     ├─────────┤     └───┘┌─┴─┐
q_3: ┤ Ry(θ_3) ├──────────┤ X ├
     └─────────┘          └───┘


In [8]:
from qiskit import transpile
from qiskit_aer import Aer
from scipy.optimize import minimize
#from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

simulator = Aer.get_backend('qasm_simulator') #quantum simulator

# Define the full quantum circuit for one data point
def create_full_circuit(data, params_values):
    qc = create_quantum_circuit(data)
    var_qc, params = create_variational_circuit(data.shape[0])
    bound_var_qc = var_qc.assign_parameters({params[i]: params_values[i] for i in range(len(params))})
    qc = qc.compose(bound_var_qc)
    #var_qc.draw()
    #qc.draw()
    qc.measure_all()
    return qc

# Cost function to minimize
def cost_function(params_values):
    total_cost = 0
    for i, data in enumerate(X_train):
        qc = create_full_circuit(data, params_values)
        transpiled_qc = transpile(qc, simulator)
        result = simulator.run(transpiled_qc, shots=1024).result()
        counts = result.get_counts()
        # Convert measurement results to binary outcomes
        predictions = (counts.get('0000', 0) + counts.get('0001', 0) >= counts.get('1110', 0) + counts.get('1111', 0))
        total_cost += (predictions - y_train[i]) ** 2
    return total_cost / len(X_train)

# Optimize the parameters
initial_params = np.random.rand(4) * np.pi
result = minimize(cost_function, initial_params, method='COBYLA')

# Optimized parameters
optimal_params = result.x
print("Optimal parameters:", optimal_params)

Optimal parameters: [ 2.16482139 -0.15508992  0.56187934  4.56133542]


In [9]:
# Evaluate on the test set
def predict(data, params_values):
    qc = create_full_circuit(data, params_values)
    transpiled_qc = transpile(qc, simulator)
    result = simulator.run(transpiled_qc, shots=1024).result()
    counts = result.get_counts()
    prediction = (counts.get('0000', 0) + counts.get('0001', 0) >= counts.get('1110', 0) + counts.get('1111', 0))
    return prediction

# Calculate accuracy
correct_predictions = 0
predictions_val = []
for i, data in enumerate(X_test):
    prediction = predict(data, optimal_params)
    predictions_val.append(prediction)
    if prediction == y_test[i]:
        correct_predictions += 1

accuracy = correct_predictions / len(X_test)
print("Test set accuracy:", accuracy)

Test set accuracy: 0.8333333333333334


In [10]:
predictions_val = np.array(predictions_val)
# now using X_test and Predictions_val

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the neural network model
model = Sequential([
    Dense(16, activation='relu', input_shape=(4,)),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_test, predictions_val, epochs=50, batch_size=8)

Epoch 1/50
4/4 [==============================] - 1s 8ms/step - loss: 0.9837 - accuracy: 0.4000
Epoch 2/50
4/4 [==============================] - 0s 5ms/step - loss: 0.9587 - accuracy: 0.4000
Epoch 3/50
4/4 [==============================] - 0s 5ms/step - loss: 0.9395 - accuracy: 0.4000
Epoch 4/50
4/4 [==============================] - 0s 5ms/step - loss: 0.9169 - accuracy: 0.4000
Epoch 5/50
4/4 [==============================] - 0s 5ms/step - loss: 0.8958 - accuracy: 0.4333
Epoch 6/50
4/4 [==============================] - 0s 9ms/step - loss: 0.8760 - accuracy: 0.4333
Epoch 7/50
4/4 [==============================] - 0s 6ms/step - loss: 0.8567 - accuracy: 0.4333
Epoch 8/50
4/4 [==============================] - 0s 6ms/step - loss: 0.8372 - accuracy: 0.4000
Epoch 9/50
4/4 [==============================] - 0s 4ms/step - loss: 0.8215 - accuracy: 0.4000
Epoch 10/50
4/4 [==============================] - 0s 4ms/step - loss: 0.8043 - accuracy: 0.4000
Epoch 11/50
4/4 [======================

In [14]:
# Compute gradients of the model's output with respect to the input image
def compute_gradients(model, input_data, target_class):
    with tf.GradientTape() as tape:
        tape.watch(input_data)
        predictions = model(input_data)
        loss = tf.reduce_mean(predictions)
    gradients = tape.gradient(loss, input_data)
    return gradients

# Find points near the decision boundary
def find_boundary_points(model, X_data, y_data):
    boundary_points = []
    for i in range(len(X_data)):
        input_data = tf.convert_to_tensor(X_data[i:i+1])
        gradients = compute_gradients(model, input_data, y_data[i])
        gradient_magnitude = tf.norm(gradients)
        #print(gradient_magnitude)
        if gradient_magnitude > 0.2:  # Adjust threshold as needed
            boundary_points.append(input_data.numpy())
    return boundary_points

# Find boundary points in the training set
boundary_points = find_boundary_points(model, X_test, predictions_val)
print("Total points: ", len(X_test))
print("Number of boundary points:", len(boundary_points))

Total points:  30
Number of boundary points: 6


In [15]:
# Perturb a point by epsilon
def perturb_point(point, epsilon):
    return np.clip(point + epsilon * np.random.randn(*point.shape), -2, 2)
# Perturb a point in opposite direction by epsilon
def perturb_point_neg(point, epsilon):
    return np.clip(point - epsilon * np.random.randn(*point.shape), -2, 2)

# Check if perturbed point generates an adversarial example
def is_adversarial(model, original_point, perturbed_point):
    original_prediction = model.predict(original_point)
    perturbed_prediction = model.predict(perturbed_point)
    return (original_prediction >= 0.5) != (perturbed_prediction >= 0.5)  # Predictions flip

# Initialize parameters
epsilon = 0.9  # Perturbation amount

# Generate adversarial examples
generated_images = []
#dictionary storing points and directions +1 or -1
pts = [] #points
dir = [] #corresponding direction
delta = 3 #parameter selecting how many new points we need to generate in our chosen direction
for point in boundary_points:
    perturbed_point = perturb_point(point, epsilon)
    perturbed_point_neg = perturb_point_neg(point, epsilon)
    if is_adversarial(model, point, perturbed_point):
        print("got_pos")
        pts.append(point)
        dir.append(1)
        generated_images.append(perturbed_point.squeeze())
        if len(generated_images) >= 10:
            break
    if is_adversarial(model, point, perturbed_point_neg):
        print("got_neg")
        pts.append(point)
        dir.append(-1)
        generated_images.append(perturbed_point.squeeze())
        if len(generated_images) >= 10:
            break

#generate more points in the selected direction
for i in range(len(pts)):
    while delta>0:
      if dir[i]==1:
        pts[i] = perturb_point(pts[i], epsilon)
      else:
        pts[i] = perturb_point_neg(pts[i],epsilon)
      if is_adversarial(model, point, pts[i]):
        generated_images.append(pts[i].squeeze())
      delta-=1

1/1 [==============================] - 0s 85ms/step
got_neg
1/1 [==============================] - 0s 90ms/step
got_pos
1/1 [==============================] - 0s 32ms/step
got_neg
1/1 [==============================] - 0s 51ms/step
got_pos
1/1 [==============================] - 0s 43ms/step
got_pos
1/1 [==============================] - 0s 32ms/step


In [ ]:
# 5 adversarial examples generated